# **Assistants API**

This lab will help you understand how to use the **Azure OpenAI Assistants API** (https://learn.microsoft.com/en-us/azure/ai-services/openai/assistants-reference), as well as the **code interpreter function** (https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/code-interpreter?tabs=python). Both will the last building blocks for your new AI-enabled database agent.

Let's load the necessary environment variables

In [1]:
import os
import json
import requests
from dotenv import load_dotenv
load_dotenv(".env")

True


Before you start, the sequence for the Assistants API includes these steps:

1.   **Create an Assistant:** Define its custom instructions and pick a model. Optionally, add files and enable tools like Code Interpreter, Retrieval, and Function calling.
2.   **Create a Thread:** A Thread represents a conversation between a user and one or many Assistants. Create a Thread when a user starts a conversation with your Assistant.
3.   **Add Messages to the Thread:** Messages from users or applications are added as Message objects to the Thread.
Messages can contain both text and files.
There’s no limit to the number of Messages you can add to Threads; context is truncated to fit the model’s context window.
4.   **Run the Assistant on the Thread:** Generate a response by calling the model and tools associated with the Assistant. Responses are added to the Thread as assistant Messages.



In [2]:
from openai import AzureOpenAI
import json
import os
import glob
should_cleanup: bool = True


client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

# Using File Search with PDF files

Let's create a new assistant with file search and code interpreter



In [3]:

assistant = client.beta.assistants.create(
  name="Insurance Submission Extractor",
  instructions="You are an AI assistant who is an expert in extracting data from insurance submissions which includes PDF and excel files. You are expected to extract the following information: 1. Named Insured 2. DBA Name 3. Coverage or Peril or Exposure  4. Policy InceptionDate 5. Policy ExpirationDate 6. StreetAddress, City, State, Zip, County details of the Property 7. TotalInsuredValue, OccupancyCode of the property/properties",
  model=os.getenv("GPT4_DEPLOYMENT_NAME"),
  tools=[{"type": "file_search"},{"type":"code_interpreter"}]
)

Upload the PDF files to the vector store and attach it to the file search tool in assistant

In [4]:
vector_store = client.beta.vector_stores.create(name="Submission Documents Vector Store")
 
# Ready the files for upload to OpenAI
file_paths = glob.glob("data/*.pdf") + glob.glob("data/*.docx")
file_streams = [open(path, "rb") for path in file_paths]
 
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

completed
FileCounts(cancelled=0, completed=2, failed=0, in_progress=0, total=2)


Create a thread using the vector store file and publish the output

In [5]:
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Extract the following attributes from the submission: NamedInsured, DBA Name, RenewalofAccountID, Coverage or Peril or Exposure, InceptionDate, ExpirationDate"
)
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)


In [6]:
import time
from IPython.display import clear_output

status = run.status
start_time = time.time()
while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = run.status
    print(f'Status: {status}')
    clear_output(wait=True)


Elapsed time: 0 minutes 5 seconds
Status: completed


In [7]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_Xmc0EcUAurKuiRMUndBfHjrI",
      "assistant_id": "asst_VULe68D0RjaG5BVyDSQoEwzw",
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [
              {
                "end_index": 548,
                "file_citation": {
                  "file_id": "assistant-cnsD1Ra4KCxZ0iNIXNRd99lT"
                },
                "start_index": 536,
                "text": "【4:0†source】",
                "type": "file_citation"
              },
              {
                "end_index": 860,
                "file_citation": {
                  "file_id": "assistant-cnsD1Ra4KCxZ0iNIXNRd99lT"
                },
                "start_index": 848,
                "text": "【4:0†source】",
                "type": "file_citation"
              },
              {
                "end_index": 914,
                "file_citation": {
                  "file_id": "assistant-cnsD1Ra4KCxZ0iNIXNR

In [8]:
data = json.loads(messages.model_dump_json(indent=2))  # Load JSON data into a Python object
output_data = data['data'][0]['content'][0]['text']['value']  # Adjusted to access the correct content

with open("data/submission.txt", "w") as file:
	file.write(json.dumps(output_data, indent=2))  # Convert list to JSON string

print("Data written to submission.txt")

Data written to submission.txt


# Using Code Interpreter with a CSV file

In [9]:
# Select the first .xlsx file from the list
xlsx_file_path = glob.glob("data/*.xlsx")[0]

message_file = client.files.create(
  file=open(xlsx_file_path, "rb"), purpose="assistants"
)

# Create a thread and attach the file to the message
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Extract the following property attributes from the SOV and provide the unique rows of the attributes : StreetAddress, City, State, Zip, County",
      # Attach the new file to the message.
      "attachments": [
        { "file_id": message_file.id, "tools": [{"type": "code_interpreter"}] }
      ],
    }
  ]
)
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

Verify the thread status and get final output of the message

In [10]:
import time
from IPython.display import clear_output

status = run.status
start_time = time.time()
while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = run.status
    print(f'Status: {status}')
    clear_output(wait=True)


Elapsed time: 0 minutes 26 seconds
Status: completed


In [11]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_099ktnS232lVnb3rfn5zJsVy",
      "assistant_id": "asst_VULe68D0RjaG5BVyDSQoEwzw",
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Here are the unique rows of the required property attributes extracted from the file:\n\n| Street Address                   | City         | State Code | Zip   | County |\n|----------------------------------|--------------|------------|-------|--------|\n| *Street Address                  | *City        | *State Code| *Zip  | County |\n| 7700 Downman Road                | New Orleans  | LA         | 70126 | NaN    |\n| 6235 Chef Menteur Hwy. front     | New Orleans  | LA         | 70126 | NaN    |\n| 6235 Chef Menteur Hwy. rear      | New Orleans  | LA         | 70126 | NaN    |\n| 3300 Preston Place A             | New Orleans  | LA         | 70131 | NaN    |\n| C3300 Preston Place B            | New Orleans  | LA  

In [12]:
data = json.loads(messages.model_dump_json(indent=2))  # Load JSON data into a Python object
output_data = data['data'][0]['content'][0]['text']['value']  # Adjusted to access the correct content

with open("data/submission.txt", "a") as file:
	file.write("\n\n" + json.dumps(output_data, indent=2) + "\n\n")  # Convert list to JSON string

print("Data written to submission.txt")

Data written to submission.txt


Create a new thread, ask the second question and get the output

In [13]:
# # Select the first .xlsx file from the list
# xlsx_file_path = glob.glob("data/*.xlsx")[0]

# message_file = client.files.create(
#   file=open(xlsx_file_path, "rb"), purpose="assistants"
# )

# # Create a thread and attach the file to the message
# thread = client.beta.threads.create(
#   messages=[
#     {
#       "role": "user",
#       "content": "Extract the following attributes from the submission: TotalInsuredValue, OccupancyCode",
#       # Attach the new file to the message.
#       "attachments": [
#         { "file_id": message_file.id, "tools": [{"type": "code_interpreter"}] }
#       ],
#     }
#   ]
# )

# Add a new user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Extract the following attributes from the submission: TotalInsuredValue, OccupancyCode"
)

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

In [14]:
status = run.status
start_time = time.time()
while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = run.status
    print(f'Status: {status}')
    clear_output(wait=True)


Elapsed time: 0 minutes 11 seconds
Status: completed


In [15]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_97RUJcjtm0VJQcXxHLvrX1Pr",
      "assistant_id": "asst_VULe68D0RjaG5BVyDSQoEwzw",
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "It appears \"Total Insured Value\" and \"Occupancy Code\" columns are not explicitly named and possibly hidden under other columns or merged cells. Given the lack of clear information, it is challenging to pinpoint these columns directly.\n\nWould you like to refine your search or provide additional details that can help identify the relevant columns? Alternatively, we can review another sheet from the Excel file, such as \"Building Updates Etc.\", to see if they contain the necessary attributes."
          },
          "type": "text"
        }
      ],
      "created_at": 1733414875,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "

In [16]:
data = json.loads(messages.model_dump_json(indent=2))  # Load JSON data into a Python object
output_data = data['data'][0]['content'][0]['text']['value']  # Adjusted to access the correct content

with open("data/submission.txt", "a") as file:
	file.write("\n\n" + json.dumps(output_data, indent=2) + "\n\n")  # Convert list to JSON string

print("Data written to submission.txt")

Data written to submission.txt


You can also check the code that was executed by the code interpreter by running the following code:


In [17]:
run_steps = client.beta.threads.runs.steps.list(
  thread_id=thread.id,
  run_id=run.id
)
print(run_steps.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "step_oL0cRxAsiXJMMlVfSBJLv9D0",
      "assistant_id": "asst_VULe68D0RjaG5BVyDSQoEwzw",
      "cancelled_at": null,
      "completed_at": 1733414876,
      "created_at": 1733414875,
      "expired_at": null,
      "failed_at": null,
      "last_error": null,
      "metadata": null,
      "object": "thread.run.step",
      "run_id": "run_x2vFNLmVmRm6S0tRBksKntKQ",
      "status": "completed",
      "step_details": {
        "message_creation": {
          "message_id": "msg_97RUJcjtm0VJQcXxHLvrX1Pr"
        },
        "type": "message_creation"
      },
      "thread_id": "thread_BTlAcFIKcvq9TobuJMAHr7u4",
      "type": "message_creation",
      "usage": {
        "completion_tokens": 94,
        "prompt_tokens": 6932,
        "total_tokens": 7026
      },
      "expires_at": null
    },
    {
      "id": "step_kpw5CxcJ0DXKyhjcZE4bGAMH",
      "assistant_id": "asst_VULe68D0RjaG5BVyDSQoEwzw",
      "cancelled_at": null,
      "completed_at": 1733414875,
  

Delete the assistant and thread once the extraction is completed

In [18]:
if should_cleanup:
    client.beta.assistants.delete(assistant.id)
    client.beta.threads.delete(thread.id)
    client.beta.vector_stores.delete(vector_store.id)